# Summary

* This is a tutorial on using Python for accessing the scBaseCount dataset hosted by the Arc Institute.
* The data can be streamed or downloaded locally.
  * For small jobs (e.g., summarizing the some metadata), streaming is recommended.
  * For large jobs (e.g., training a model), downloading is recommended.
* See the [README](README.md#metadata) for a description of the obs metadata.


# Setup

### Installation

If needed, install the necessary dependencies.

You can use the [conda environment](../conda_envs/python.yml) provided in this git repository.

# Load packages

In [ ]:
import os
import pandas as pd
import scanpy as sc
import pyarrow.dataset as ds
import gcsfs

In [ ]:
# initialize GCS file system for reading data from GCS
fs = gcsfs.GCSFileSystem()

# Data location

In [ ]:
# GCS bucket path
gcs_base_path = "gs://arc-scbasecount/2025-02-25/"

In [ ]:
# STARsolo feature type
feature_type = "GeneFull_Ex50pAS"

# List available files

Let's see what we have to work with!

First, load some helper code.

In [ ]:
# helper function to list files 
def get_file_table(gcs_base_path: str, target: str=None, endswith: str=None):
    files = fs.glob("/".join([gcs_base_path.rstrip("/"), "**"]))
    if target:
        files = [f for f in files if os.path.basename(f) == target]
    else:
        files = [f for f in files if f.endswith(endswith)]
    file_list = []
    for f in files:
        file_list.append(f.split("/")[-2:-1] + [f])
    return pd.DataFrame(file_list, columns=["organism", "file_path"])

## Parquet files

* Contain the obs metadata
* These can be read efficiently with [pyarrow](https://arrow.apache.org/docs/python/index.html)
  * We will read in via pyarrow and convert to pandas

In [27]:
# set the path to the metadata files
gcs_path = "/".join([gcs_base_path.rstrip("/"), "metadata", feature_type])
gcs_path

'gs://arc-scbasecount/2025-02-25/metadata/GeneFull_Ex50pAS'

### List per-sample metadata files

Per-sample (SRX accession) metadata (e.g., tissue)

In [28]:
# list files
sample_pq_files = get_file_table(gcs_path, "sample_metadata.parquet")
print(sample_pq_files.shape)
sample_pq_files.head()

(21, 2)


,organism,file_path
0,Arabidopsis_thaliana,arc-scbasecount/2025-02-25/metadata/GeneFull_E...
1,Bos_taurus,arc-scbasecount/2025-02-25/metadata/GeneFull_E...
2,Caenorhabditis_elegans,arc-scbasecount/2025-02-25/metadata/GeneFull_E...
3,Callithrix_jacchus,arc-scbasecount/2025-02-25/metadata/GeneFull_E...
4,Danio_rerio,arc-scbasecount/2025-02-25/metadata/GeneFull_E...


**Notes:**

* As you can see, the files are organized by `feature_type` (STARsolo output type) and `organism`

### List per-obs metadata files

Per-observation (cell) metadata

In [29]:
# list files
obs_pq_files = get_file_table(gcs_path, "obs_metadata.parquet")
print(obs_pq_files.shape)
obs_pq_files.head()

(21, 2)


,organism,file_path
0,Arabidopsis_thaliana,arc-scbasecount/2025-02-25/metadata/GeneFull_E...
1,Bos_taurus,arc-scbasecount/2025-02-25/metadata/GeneFull_E...
2,Caenorhabditis_elegans,arc-scbasecount/2025-02-25/metadata/GeneFull_E...
3,Callithrix_jacchus,arc-scbasecount/2025-02-25/metadata/GeneFull_E...
4,Danio_rerio,arc-scbasecount/2025-02-25/metadata/GeneFull_E...


## h5ad files 

* Contain count matrices and per-obs metadata

In [30]:
# set the path
gcs_path = "/".join([gcs_base_path.rstrip("/"), "h5ad", feature_type])
gcs_path

'gs://arc-scbasecount/2025-02-25/h5ad/GeneFull_Ex50pAS'

In [31]:
# list files
h5ad_files = get_file_table(gcs_path, endswith=".h5ad")
print(h5ad_files.shape)
h5ad_files.head()

(30387, 2)


,organism,file_path
0,Arabidopsis_thaliana,arc-scbasecount/2025-02-25/h5ad/GeneFull_Ex50p...
1,Arabidopsis_thaliana,arc-scbasecount/2025-02-25/h5ad/GeneFull_Ex50p...
2,Arabidopsis_thaliana,arc-scbasecount/2025-02-25/h5ad/GeneFull_Ex50p...
3,Arabidopsis_thaliana,arc-scbasecount/2025-02-25/h5ad/GeneFull_Ex50p...
4,Arabidopsis_thaliana,arc-scbasecount/2025-02-25/h5ad/GeneFull_Ex50p...


# Explore the per-sample metadata

### Just human samples

In [32]:
# get the per-sample metadata file path
infile = sample_pq_files[sample_pq_files["organism"] == "Homo_sapiens"]["file_path"].values[0]
infile

'arc-scbasecount/2025-02-25/metadata/GeneFull_Ex50pAS/Homo_sapiens/sample_metadata.parquet'

In [33]:
# load the metadata
sample_metadata = ds.dataset(infile, filesystem=fs, format="parquet").to_table().to_pandas()
print(sample_metadata.shape)
sample_metadata.head()

(16077, 14)


,entrez_id,srx_accession,file_path,obs_count,lib_prep,tech_10x,cell_prep,organism,tissue,disease,perturbation,cell_line,czi_collection_id,czi_collection_name
0,29110018,ERX11148735,gs://arc-scbasecount/2025-02-25/h5ad/GeneFull_...,747,10x_Genomics,3_prime_gex,single_cell,Homo sapiens,skin of body,normal,surplus skin from breast reconstruction surgery,not applicable,73f82ac8-15cc-4fcd-87f8-5683723fce7f,Developmental cell programs are co-opted in in...
1,29110027,ERX11148744,gs://arc-scbasecount/2025-02-25/h5ad/GeneFull_...,2379,10x_Genomics,3_prime_gex,single_cell,Homo sapiens,skin of body,normal,treated with dispase II and collagenase for ce...,keratinocyte CD49f-,73f82ac8-15cc-4fcd-87f8-5683723fce7f,Developmental cell programs are co-opted in in...
2,29110026,ERX11148743,gs://arc-scbasecount/2025-02-25/h5ad/GeneFull_...,2316,10x_Genomics,3_prime_gex,single_cell,Homo sapiens,skin of body,normal,treated with dispase II and collagenase for ce...,epidermal myeloid cells,73f82ac8-15cc-4fcd-87f8-5683723fce7f,Developmental cell programs are co-opted in in...
3,29110023,ERX11148740,gs://arc-scbasecount/2025-02-25/h5ad/GeneFull_...,2907,10x_Genomics,3_prime_gex,single_cell,Homo sapiens,skin of body,normal,skin collected from breast reconstruction surgery,not specified,73f82ac8-15cc-4fcd-87f8-5683723fce7f,Developmental cell programs are co-opted in in...
4,29110015,ERX11148732,gs://arc-scbasecount/2025-02-25/h5ad/GeneFull_...,4082,10x_Genomics,3_prime_gex,single_cell,Homo sapiens,skin of body,normal,treated with dispase II and collagenase,not_applicable,73f82ac8-15cc-4fcd-87f8-5683723fce7f,Developmental cell programs are co-opted in in...


In [47]:
# All human?
sample_metadata["organism"].value_counts()

organism
Homo sapiens    16077
Name: count, dtype: int64

In [ ]:
# 10X library prep methods
sample_metadata["tech_10x"].value_counts()

In [ ]:
# cell prep method
sample_metadata["cell_prep"].value_counts()

### All organisms

Let's scale up to everything!

In [48]:
# Read in the metadata for all organisms
sample_metadata = []
for i,row in sample_pq_files.iterrows():
    sample_metadata.append(
        ds.dataset(row["file_path"], filesystem=fs, format="parquet").to_table().to_pandas()
    )
sample_metadata = pd.concat(sample_metadata)

print(f"Number of samples: {sample_metadata.shape[0]}")
sample_metadata.head()

Number of samples: 30387


,entrez_id,srx_accession,file_path,obs_count,lib_prep,tech_10x,cell_prep,organism,tissue,disease,perturbation,cell_line,czi_collection_id,czi_collection_name
0,24123125,SRX17302366,gs://arc-scbasecount/2025-02-25/h5ad/GeneFull_...,9036,10x_Genomics,3_prime_gex,single_cell,Arabidopsis thaliana,other,not specified,"BL (Brassinolide), 100nM, 0.5 hours post-treat...",WT Col-0,None,None
1,24123140,SRX17302381,gs://arc-scbasecount/2025-02-25/h5ad/GeneFull_...,14317,10x_Genomics,3_prime_gex,single_cell,Arabidopsis thaliana,other,not specified,"control treatment, age: 7 days",WT Col-0,None,None
2,24123142,SRX17302383,gs://arc-scbasecount/2025-02-25/h5ad/GeneFull_...,20075,10x_Genomics,3_prime_gex,single_cell,Arabidopsis thaliana,other,unsure,control,unsure,None,None
3,26626960,SRX19366049,gs://arc-scbasecount/2025-02-25/h5ad/GeneFull_...,7539,10x_Genomics,3_prime_gex,single_cell,Arabidopsis thaliana,other,unsure,mock treatment (control group),not applicable,None,None
4,26626958,SRX19366047,gs://arc-scbasecount/2025-02-25/h5ad/GeneFull_...,7703,10x_Genomics,3_prime_gex,single_cell,Arabidopsis thaliana,other,none,mock treatment; 2 µM RALF1 peptide for 2 hours,none,None,None


In [49]:
# cells
print(f"Obs count: {sample_metadata['obs_count'].sum()}")

Obs count: 233686476


In [50]:
# samples per organism
sample_metadata["organism"].value_counts()

organism
Homo sapiens               16077
Mus musculus               12212
Macaca mulatta               587
Danio rerio                  458
Sus scrofa                   195
Drosophila melanogaster      181
Arabidopsis thaliana         175
Gallus gallus                102
Heterocephalus glaber         79
Caenorhabditis elegans        52
Pan troglodytes               49
Bos taurus                    48
Oryctolagus cuniculus         34
Zea mays                      33
Oryza sativa                  31
Callithrix jacchus            24
Ovis aries                    20
Equus caballus                11
Solanum lycopersicum          10
Schistosoma mansoni            7
Gorilla gorilla                2
Name: count, dtype: int64

In [51]:
# Get stats for primates
primate_species = ["Homo sapiens", "Pan troglodytes", "Gorilla gorilla", "Macaca mulatta"]

for species in primate_species:
    species_samples = sample_metadata[sample_metadata["organism"] == species]
    cell_count = species_samples["obs_count"].sum()
    sample_count = species_samples.shape[0]
    
    if sample_count > 0:
        avg_cells = cell_count / sample_count
        print(f"\n{species}:")
        print(f"Number of cells: {cell_count:,}")
        print(f"Number of samples: {sample_count:,}")
        print(f"Average cells per sample: {avg_cells:.1f}")



Homo sapiens:
Number of cells: 118,671,014
Number of samples: 16,077
Average cells per sample: 7381.4

Pan troglodytes:
Number of cells: 312,701
Number of samples: 49
Average cells per sample: 6381.7

Gorilla gorilla:
Number of cells: 9,351
Number of samples: 2
Average cells per sample: 4675.5

Macaca mulatta:
Number of cells: 4,057,401
Number of samples: 587
Average cells per sample: 6912.1


In [ ]:
# Get stats for all organisms and plot histogram
import matplotlib.pyplot as plt
import numpy as np

# Calculate stats per species
species_stats = []
for species in sample_metadata["organism"].unique():
    species_samples = sample_metadata[sample_metadata["organism"] == species]
    cell_count = species_samples["obs_count"].sum()
    sample_count = species_samples.shape[0]
    
    if sample_count > 0:
        avg_cells = cell_count / sample_count
        species_stats.append({
            'species': species,
            'cell_count': cell_count,
            'sample_count': sample_count,
            'avg_cells': avg_cells
        })

# Convert to DataFrame for plotting
stats_df = pd.DataFrame(species_stats)
stats_df = stats_df.sort_values('cell_count', ascending=False)

# Create histogram with log scale
plt.figure(figsize=(12, 6))
plt.bar(range(len(stats_df)), np.log10(stats_df['cell_count']))
plt.xticks(range(len(stats_df)), stats_df['species'], rotation=45, ha='right')
plt.ylabel('log10(Number of Cells)')
plt.title('Cell Counts by Species (Log Scale)')
plt.tight_layout()
plt.savefig('cell_counts_by_species.png')
plt.show()


In [ ]:
# tech_10x
sample_metadata["tech_10x"].value_counts()

In [ ]:
# samples associated with czi collections
czi_sample_count = sample_metadata[~sample_metadata["czi_collection_id"].isna()].shape[0]
print(f"Samples associated with CZI collections: {czi_sample_count}")

In [ ]:
# check that the file paths point to existing h5ad files (assumes you have gsutil  installed)
!which gsutil && gsutil ls {sample_metadata["file_path"].values[0]}

# Explore the per-obs metadata

* `obs` ≃ cell

In [ ]:
# The list of metadata files per organism
obs_pq_files

In [ ]:
# let's read in the metadata for a single organism
target_organism = "Bos_taurus"

In [ ]:
# extract the file path
infile = obs_pq_files[obs_pq_files["organism"] == target_organism]["file_path"].values[0]

In [ ]:
# read in the first 100000 rows
obs_metadata = ds.dataset(infile, filesystem=fs, format="parquet").head(100000).to_pandas()
print(obs_metadata.shape)
obs_metadata.head()

In [ ]:
# distribution of gene counts
obs_metadata["gene_count"].describe()

In [ ]:
# distribution of umi counts
obs_metadata["umi_count"].describe()

## Get per-obs metadata for specific samples

Method:

1. Query the sample metadata
2. Use the filtered sample metadata to query the cell metadata

#### Filter sample metadata

Let's get all sheep and horse samples with `obs_count > 10000`

In [ ]:
target_organisms = ["Ovis aries", "Equus caballus"]
obs_count_cutoff = 10000

In [ ]:
# get the target samples
target_samples = sample_metadata[(sample_metadata["organism"].isin(target_organisms)) & (sample_metadata["obs_count"] > obs_count_cutoff)]
print(target_samples.shape)
target_samples.head()

In [ ]:
# filter the obs metadata
target_orgs = [x.replace(" ", "_") for x in target_samples["organism"].unique().tolist()]
target_obs_files = obs_pq_files[obs_pq_files["organism"].isin(target_orgs)]
target_obs_files

In [ ]:
# read in the obs metadata
obs_metadata = []
for i,row in target_obs_files.iterrows():
    obs_metadata.append(
        ds.dataset(row["file_path"], filesystem=fs, format="parquet").to_table().to_pandas()
    )
obs_metadata = pd.concat(obs_metadata)

# merge with the target samples
obs_metadata = target_samples.merge(obs_metadata, left_on="srx_accession", right_on="SRX_accession")

print(obs_metadata.shape)
obs_metadata.head()

In [ ]:
# gene_count distribution per sample
obs_metadata.groupby(["organism", "srx_accession"])["gene_count"].describe()

In [ ]:
# umi_count distribution per sample
obs_metadata.groupby(["organism", "srx_accession"])["umi_count"].describe()

# Read h5ad files

### Example: select marmoset samples

In [ ]:
# get the target samples
query = (sample_metadata["organism"] == "Callithrix jacchus") & (sample_metadata["obs_count"] < 3000)
target_samples = sample_metadata[query].head(n=3)
target_samples

In [ ]:
# read in the anndata for those samples
adata = []
for infile in target_samples["file_path"].tolist():
    print(infile)
    with fs.open(infile, 'rb') as f:
        adata.append(sc.read_h5ad(f))

# combine anndata objects
adata = sc.concat(adata)
adata

In [34]:
## Download Homo sapiens data for discrete diffusion transformer

# First, let's get human samples and pick one for preview
human_samples = sample_metadata[sample_metadata["organism"] == "Homo sapiens"]
print(f"Total human samples: {human_samples.shape[0]:,}")
print(f"Total human cells: {human_samples['obs_count'].sum():,}")

# Check available columns in the metadata
print(f"\nAvailable columns: {list(human_samples.columns)}")

# Sort by cell count and pick a medium-sized sample for preview
human_samples_sorted = human_samples.sort_values('obs_count', ascending=False)
preview_sample = human_samples_sorted.iloc[len(human_samples_sorted)//2]  # Pick middle sample

print(f"\nPreview sample selected:")
print(f"SRX ID: {preview_sample['srx_accession']}")
print(f"Cell count: {preview_sample['obs_count']:,}")
print(f"File path: {preview_sample['file_path']}")
print(f"Tissue: {preview_sample['tissue']}")
print(f"10X tech: {preview_sample['tech_10x']}")
print(f"Cell prep: {preview_sample['cell_prep']}")
if 'czi_collection_name' in human_samples.columns:
    print(f"Study/Collection: {preview_sample['czi_collection_name']}")


Total human samples: 16,077
Total human cells: 118,671,014

Available columns: ['entrez_id', 'srx_accession', 'file_path', 'obs_count', 'lib_prep', 'tech_10x', 'cell_prep', 'organism', 'tissue', 'disease', 'perturbation', 'cell_line', 'czi_collection_id', 'czi_collection_name']

Preview sample selected:
SRX ID: SRX21215694
Cell count: 5,829
File path: gs://arc-scbasecount/2025-02-25/h5ad/GeneFull_Ex50pAS/Homo_sapiens/SRX21215694.h5ad
Tissue: blood
10X tech: multiome
Cell prep: single_nucleus
Study/Collection: None


In [39]:
# Download the preview sample to inspect it
preview_gcs_path = preview_sample['file_path']

print(f"Downloading preview sample: {preview_sample['srx_accession']}")
print(f"From: {preview_gcs_path}")

try:
    # Use gcsfs to download the file
    output_file = f"{preview_sample['srx_accession']}.h5ad"
    fs.get(preview_gcs_path, output_file)
    print(f"✅ Successfully downloaded preview data to {output_file}")
    
    # Load and inspect the downloaded file
    preview_adata = sc.read_h5ad(output_file)
    
    # Basic data info
    print(f"\nData shape: {preview_adata.shape}")
    print(f"Genes: {preview_adata.n_vars:,}")
    print(f"Cells: {preview_adata.n_obs:,}")
    
    # Show one cell row (gene expression profile)
    print(f"\n--- Sample Cell Row (first cell, first 10 genes) ---")
    print("Cell ID:", preview_adata.obs_names[0])
    
    # Handle both sparse and dense matrices
    if hasattr(preview_adata.X, 'toarray'):
        first_cell = preview_adata.X[0, :10].toarray().flatten()
    else:
        first_cell = preview_adata.X[0, :10]
    
    gene_names = preview_adata.var_names[:10].tolist()
    
    print("\nGene expression counts:")
    for gene, count in zip(gene_names, first_cell):
        print(f"  {gene}: {int(count)}")
    
    # Show cell metadata
    print(f"\n--- Cell Metadata (first cell) ---")
    if len(preview_adata.obs.columns) > 0:
        print(preview_adata.obs.iloc[0])
    else:
        print("No cell metadata available")
    
    # Data statistics for transformer training
    print(f"\n--- Data Statistics for Transformer Training ---")
    if hasattr(preview_adata.X, 'toarray'):
        X_dense = preview_adata.X.toarray()
        sparsity = (X_dense == 0).sum() / X_dense.size * 100
        max_count = X_dense.max()
        mean_counts_per_cell = X_dense.sum(axis=1).mean()
        unique_values = len(np.unique(X_dense))
    else:
        sparsity = (preview_adata.X == 0).sum() / preview_adata.X.size * 100
        max_count = preview_adata.X.max()
        mean_counts_per_cell = preview_adata.X.sum(axis=1).mean()
        unique_values = len(np.unique(preview_adata.X))
    
    print(f"Sparsity: {sparsity:.1f}% zeros")
    print(f"Max count: {max_count}")
    print(f"Mean counts per cell: {mean_counts_per_cell:.1f}")
    print(f"Unique count values: {unique_values}")
    
except Exception as e:
    print(f"❌ Failed to download/read data: {str(e)}")


From: gs://arc-scbasecount/2025-02-25/h5ad/GeneFull_Ex50pAS/Homo_sapiens/SRX21215694.h5ad
✅ Successfully downloaded preview data to SRX21215694.h5ad

Data shape: (5829, 36601)
Genes: 36,601
Cells: 5,829

--- Sample Cell Row (first cell, first 10 genes) ---
Cell ID: AAACAGCCACATGCTA

Gene expression counts:
  ENSG00000243485: 0
  ENSG00000237613: 0
  ENSG00000186092: 0
  ENSG00000238009: 0
  ENSG00000239945: 0
  ENSG00000239906: 0
  ENSG00000241860: 0
  ENSG00000241599: 0
  ENSG00000286448: 0
  ENSG00000236601: 0

--- Cell Metadata (first cell) ---
gene_count              3019
umi_count             7539.0
SRX_accession    SRX21215694
Name: AAACAGCCACATGCTA, dtype: object

--- Data Statistics for Transformer Training ---
Sparsity: 92.9% zeros
Max count: 4535.0
Mean counts per cell: 6941.9
Unique count values: 816


In [46]:
# Preview one row of the human samples dataframe
print("\n--- Sample Metadata Row ---")
print(human_samples.iloc[0])

# Download and examine one sample file
sample_path = "SRX21215694.h5ad"
print(f"\n📥 Reading file: {sample_path}")

try:
    # Read the local file
    preview_adata = sc.read_h5ad(sample_path)
    
    print(f"\n--- Dataset Overview ---")
    print(f"Number of cells: {preview_adata.n_obs:,}")
    print(f"Number of genes: {preview_adata.n_vars:,}")
    
    # Handle sparse matrix size calculation
    if hasattr(preview_adata.X, 'data'):
        size_gb = preview_adata.X.data.nbytes / 1e9
    else:
        size_gb = preview_adata.X.nbytes / 1e9
    print(f"Total size in memory: {size_gb:.1f} GB")
    
    # Show available metadata fields
    print(f"\n--- Available Metadata Fields ---")
    print("Cell metadata columns:", list(preview_adata.obs.columns))
    print("Gene metadata columns:", list(preview_adata.var.columns))
    
    # Save one row of gene expression data to CSV
    print("\n--- Saving Sample Gene Expression Row ---")
    if hasattr(preview_adata.X, 'toarray'):
        first_row = preview_adata.X[0].toarray()[0]
    else:
        first_row = preview_adata.X[0]
    
    # Create dataframe with gene names and counts
    import pandas as pd
    gene_counts = pd.DataFrame({
        'gene': preview_adata.var_names,
        'count': first_row
    })
    
    # Save to CSV
    output_file = 'first_cell_gene_counts.csv'
    gene_counts.to_csv(output_file, index=False)
    print(f"✓ Saved gene expression data to {output_file}")
    
    # Preview first few rows
    print("\nPreview of saved data:")
    print(gene_counts.head())
except Exception as e:
    print(f"❌ Failed to read file: {str(e)}")



--- Sample Metadata Row ---
entrez_id                                                       29110018
srx_accession                                                ERX11148735
file_path              gs://arc-scbasecount/2025-02-25/h5ad/GeneFull_...
obs_count                                                            747
lib_prep                                                    10x_Genomics
tech_10x                                                     3_prime_gex
cell_prep                                                    single_cell
organism                                                    Homo sapiens
tissue                                                      skin of body
disease                                                           normal
perturbation             surplus skin from breast reconstruction surgery
cell_line                                                 not applicable
czi_collection_id                   73f82ac8-15cc-4fcd-87f8-5683723fce7f
czi_collection_name   

In [38]:
# After confirming the preview data looks good, run this to download ALL human data
# WARNING: This will download ~16,077 files - make sure you have sufficient disk space!

def download_all_human_data(batch_size=100, start_idx=0):
    """
    Download all human samples in batches for discrete diffusion transformer training
    
    Args:
        batch_size: Number of files to download per batch 
        start_idx: Starting index (useful for resuming interrupted downloads)
    """
    
    # Create directory for human data
    import os
    os.makedirs("human_data", exist_ok=True)
    
    print(f"📊 Total human samples to download: {len(human_samples):,}")
    print(f"💾 Estimated total size: ~{len(human_samples) * 50}MB (rough estimate)")
    print(f"🔄 Downloading in batches of {batch_size}")
    
    # Get list of files to download
    download_list = []
    for idx, (_, row) in enumerate(human_samples.iterrows()):
        if idx < start_idx:
            continue
            
        src_path = row['file_path'] 
        dst_filename = f"human_data/{row['srx_accession']}.h5ad"
        download_list.append((src_path, dst_filename, row['srx_accession'], row['obs_count']))
    
    # Download in batches
    failed_downloads = []
    successful_downloads = 0
    
    for i in range(0, len(download_list), batch_size):
        batch = download_list[i:i+batch_size]
        batch_num = i // batch_size + 1
        total_batches = (len(download_list) + batch_size - 1) // batch_size
        
        print(f"\n🚀 Batch {batch_num}/{total_batches} - Downloading {len(batch)} files...")
        
        for src_path, dst_filename, srx_id, cell_count in batch:
            # Skip if already downloaded
            if os.path.exists(dst_filename):
                print(f"⏭️  Skipping {srx_id} (already exists)")
                successful_downloads += 1
                continue
                
            # Download file
            result = subprocess.run([
                'gsutil', 'cp', src_path, dst_filename
            ], capture_output=True, text=True)
            
            if result.returncode == 0:
                print(f"✅ {srx_id} ({cell_count:,} cells)")
                successful_downloads += 1
            else:
                print(f"❌ Failed: {srx_id} - {result.stderr.strip()}")
                failed_downloads.append((srx_id, src_path, result.stderr))
        
        # Progress update
        remaining = len(download_list) - i - len(batch)
        print(f"📈 Progress: {successful_downloads}/{len(download_list)} successful, {remaining} remaining")
    
    print(f"\n🎉 Download Summary:")
    print(f"✅ Successful: {successful_downloads}")
    print(f"❌ Failed: {len(failed_downloads)}")
    
    if failed_downloads:
        print(f"\n💔 Failed downloads:")
        for srx_id, path, error in failed_downloads[:5]:  # Show first 5 failures
            print(f"  {srx_id}: {error.strip()}")
        if len(failed_downloads) > 5:
            print(f"  ... and {len(failed_downloads) - 5} more")
    
    return successful_downloads, failed_downloads

# Uncomment the line below to start downloading ALL human data
# successful, failed = download_all_human_data(batch_size=50)

print("⚠️  Uncomment the last line above to start downloading all human data")
print("💡 Recommended: Start with a small batch to test your setup first")


⚠️  Uncomment the last line above to start downloading all human data
💡 Recommended: Start with a small batch to test your setup first


In [ ]:
# Additional data processing tips for discrete diffusion transformers

print("🧬 Data Processing Tips for Discrete Diffusion Transformers:")
print("\n1. Data Format:")
print("   - Each h5ad file contains a sparse count matrix (cells × genes)")
print("   - Gene expression counts are typically integers (0, 1, 2, ...)")
print("   - Perfect for discrete diffusion models!")

print("\n2. Preprocessing Pipeline:")
print("   - Filter low-quality cells (low gene count, high mitochondrial %)")
print("   - Filter low-expressed genes")
print("   - Normalize/scale if needed (depends on your model)")
print("   - Consider batching samples by study or experiment")

print("\n3. Memory Management:")
print("   - Files are in sparse format - convert to dense for training")
print("   - Consider processing in chunks if memory is limited")
print("   - Use generators/data loaders for training")

print("\n4. Tokenization Ideas:")
print("   - Directly use count values as discrete tokens")
print("   - Bin counts into discrete levels (0, 1-5, 6-10, 11+)")
print("   - Use vocabulary of most variable genes")

if 'preview_adata' in locals():
    print(f"\n📊 Your Preview Data Stats:")
    print(f"   - Shape: {preview_adata.shape[0]:,} cells × {preview_adata.shape[1]:,} genes")
    print(f"   - Sparsity: {(preview_adata.X == 0).sum() / preview_adata.X.size * 100:.1f}% zeros")
    print(f"   - Max count: {preview_adata.X.max()}")
    print(f"   - Mean counts per cell: {preview_adata.X.sum(axis=1).mean():.1f}")

print(f"\n🚀 Ready to download {len(human_samples):,} human samples!")
print("Run the cells above to proceed with your discrete diffusion transformer dataset.")


In [ ]:
# number of obs per SRX accession
adata.obs["SRX_accession"].value_counts()

In [ ]:
# add per-sample metadata to the anndata object
adata.obs = adata.obs.reset_index().merge(
    target_samples, left_on="SRX_accession", right_on="srx_accession", how="inner"
)
adata.obs.head()

# Downloading files

You can use [gsutil](https://cloud.google.com/storage/docs/gsutil) to download any of the files in the bucket
and work with them locally. 

Please be considerate to the [cost of egress](https://cloud.google.com/storage/pricing) when download the data from Google Cloud Storage.

For example:

```bash
gsutil cp gs://arc-scbasecount/2025-02-25/h5ad/Homo_sapiens/ERX4319106.h5ad .
```

For large data transfers, it is better to use `gsutil rsync`:

```bash
gsutil rsync gs://arc-scbasecount/2025-02-25/h5ad/Callithrix_jacchus/ .
```

***

# Session Info

In [ ]:
!conda list